In [ ]:
import re
import pandas as pd
import os
import requests
import time
import math
import requests
from bs4 import BeautifulSoup

In [ ]:
tax_df = pd.read_csv('property_tax_report_2019.csv')

In [ ]:
bca_properties_details_df = pd.DataFrame()
total_properties = len(tax_df)

num_records_per_csv = math.ceil(total_properties/20)
current_file = 0
part_csv_files_count = math.ceil(total_properties / num_records_per_csv)

index_to_start_from = 0
for index in range(0, part_csv_files_count):
    tmp_csv_file_name = '{}/tmp_{}.csv'.format(os.getcwd(), index)
    if os.path.exists(tmp_csv_file_name):
        index_to_start_from += num_records_per_csv
        current_file = index + 1
    else:
        index_to_start_from = index * num_records_per_csv
        break

tax_df_final = tax_df[index_to_start_from:]
total_properties_to_process = len(tax_df)
actual_index = 0

for index, property in tax_df_final.iterrows():
    if 'ADDRESS' in property:
        if property['ADDRESS'] is not None:
            continue
            
    if property['PID'] is None:
        continue
        
    property_id = str(property['PID'])
    print(property_id)
    pid_plain = property_id.replace('-', '')
    print(pid_plain)

    url = 'https://www.bcassessment.ca/Property/Search/GetByPid/{}?PID={}&_={}'.format(pid_plain, pid_plain, time_stamp)
    
    address = ''
    try:
        resp = requests.get(url=url)
        data = resp.json()

        data_address = data['aaData'][0]
        if len(data_address) >= 3:
            address = ' '.join(data_address[2].split(' ')[0:-1])
    except Exception as e:
            print(e)
            continue
            
    property['ADDRESS'] = address
    bca_properties_details_df = bca_properties_details_df.append(property)
    print('{}==={}==={}'.format(property_id, pid_plain, address))
    actual_index += 1
    print('{}=of={}'.format(actual_index, total_data))
    # try to save part results into the temp csv
    if ((actual_index + 1) % num_records_per_csv) == 0:
        tmp_file_name = 'tmp_{}.csv'.format(current_file)
        bca_properties_details_df.to_csv(tmp_file_name)
        current_file += 1
        bca_properties_details_df = pd.DataFrame()
        print('File with {} records {} out of {}'.format(tmp_file_name, (actual_index + 1), total_properties_to_process))
    actual_index += 1

# Write the remaining DF to storage
if len(bca_properties_details_df) > 0:
    tmp_file_name = 'tmp_{}.csv'.format(current_file)
    bca_properties_details_df.to_csv(tmp_file_name)

#### Merge all the files to create property_tax_report_with_address_2019.csv

In [ ]:
total_properties = len(tax_df)

num_records_per_csv = math.ceil(total_properties/20)
current_file = 0
part_csv_files_count = math.ceil(total_properties / num_records_per_csv)

bca_properties_with_address_df = pd.DataFrame()
for index in range(0, part_csv_files_count):
    bca_properties_with_address_df = pd.concat(
        [bca_properties_with_address_df, pd.read_csv('tmp_{}.csv'.format(index))], 
        ignore_index=False
    )

bca_properties_with_address_df.to_csv('property_tax_report_with_address_2019.csv')

#### GET HPI BENCHMARK PRICES FROM REBGV

In [ ]:
years = [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2018, 2019]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
month_map = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'};
areas = ['vancouver_east', 'vancouver_west']

hpi_benchmark_prices_df = pd.DataFrame()
for area in areas:
    for year in years:
        for month in months:
            url = 'https://www.rebgv.org/market-watch/MLS-HPI-home-price-comparison.hpi.{}.all.{}-{}-1.html'.format(area, year, month)
            request = requests.get(url)
            soup = BeautifulSoup(request.text, "html.parser")

            table = soup.findAll("table", id="hpiTableArea")
            table_rows = table[0].findAll("tr")

            index = 0
            for tr in table_rows:
                if index == 0:
                    index += 1
                    continue
                all_td = tr.findAll("td")
                print('{} {}'.format(year, month_map.get(month)))
                print('{} {}'.format(all_td[-1].text, all_td[0].text))
                hpi_benchmark_prices_df = hpi_benchmark_prices_df.append({'month': month_map.get(month), 
                                'year': year,
                                'area': area,
                                'property_type': all_td[-1].text,
                                'benchmark_price_string': all_td[0].text,
                                'price_index': all_td[1].text}, ignore_index=True)
                

In [17]:
hpi_benchmark_prices_df['benchmark_price'] = hpi_benchmark_prices_df['benchmark_price_string'].map(lambda x: x.replace(',', '')[1:])
hpi_benchmark_prices_df['benchmark_price'] = hpi_benchmark_prices_df['benchmark_price'].astype(int)
hpi_benchmark_prices_df.to_csv('HPI_benchmark_prices_rebgv.csv')
